In [33]:
import os
import pandas as pd
from dotenv import load_dotenv
from googleapiclient.discovery import build

In [41]:
load_dotenv()
api_keys = os.getenv("YOUTUBE_API_KEYS").split(',')

In [35]:
def build_youtube_client(api_key):
    return build('youtube', 'v3', developerKey=api_key)

In [36]:
def rotate_api_key(current_index):
    return (current_index + 1) % len(api_keys)

In [42]:
def get_comments(video_id, max_results=10000):
    comments = []
    current_index = 0
    youtube = build_youtube_client(api_keys[current_index])
    next_page_token = None
    
    while len(comments) < max_results:
        try:
            request = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                maxResults=min(100, max_results - len(comments)),
                pageToken=next_page_token,
                textFormat="plainText"
            )
            response = request.execute()
            
            for item in response.get("items", []):
                comment = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
                comments.append(comment)
            
            next_page_token = response.get("nextPageToken")
            
            if not next_page_token:
                break 

        except Exception as e:
            print(f"Error: {e}")
            current_index = rotate_api_key(current_index)
            youtube = build_youtube_client(api_keys[current_index])
            break
    df = pd.DataFrame(comments, columns=["Comment"])
    return df

In [ ]:
if __name__ == "__main__":
    video_id = "zzwRbKI2pn4"  
    comments_df = get_comments(video_id)
    print(comments_df)